<a href="https://colab.research.google.com/github/Suryabhan23/Assigment--1/blob/main/IndigoAirlinesFeedback.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files


In [2]:
uploaded = files.upload()

Saving indigo_meal_feedback_case_study.csv to indigo_meal_feedback_case_study.csv


In [3]:

import pandas as pd

df = pd.read_csv("indigo_meal_feedback_case_study.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   feedback_id       100 non-null    object 
 1   customer_id       100 non-null    object 
 2   meal_type         100 non-null    object 
 3   route             100 non-null    object 
 4   taste_rating      96 non-null     float64
 5   freshness_rating  100 non-null    int64  
 6   packaging_rating  100 non-null    int64  
 7   complaint_flag    98 non-null     object 
 8   meal_provider     100 non-null    object 
dtypes: float64(1), int64(2), object(6)
memory usage: 7.2+ KB


Step 1: Prepare the Data

In [11]:
#Remove duplicate entries based on feedback_id
clean_ticket = df.drop_duplicates(subset='feedback_id', keep='first')
print(clean_ticket)

#Ensure that all values in meal_type are in title case (e.g., Veg, Non-Veg)

df['meal_type'] = df['meal_type'].str.title()
print(df['meal_type'].head())

# Convert all values in the route column to uppercase
df['route'] = df['route'].str.upper()
print(df['route'])


#4. Fix values in `complaint_flag` column:
   # - Standardize misspelled entries like `"yess"` → `"Yes"`, `"noo"` → `"No"`
   # - Fill missing values with `"Not Available"`
df['complaint_flag'] = df['complaint_flag'].replace({ 'yess' : 'Yes','noo':'No'}).fillna('Not Available')
print(df['complaint_flag'])

#Fill missing values in taste_rating with a neutral default (use 3)

df['taste_rating']= df['taste_rating'].fillna(3)
print(df['taste_rating'])

   feedback_id customer_id meal_type    route  taste_rating  freshness_rating  \
0    FDBK-1000    CUST-101       Veg  BOM-HYD           1.0                 1   
1    FDBK-1001    CUST-108       Veg  DEL-BLR           1.0                 5   
2    FDBK-1002    CUST-107       Veg  MAA-CCU           2.0                 5   
3    FDBK-1003    CUST-100   Non-Veg  BOM-HYD           2.0                 4   
4    FDBK-1004    CUST-102   Non-Veg  DEL-BLR           1.0                 3   
..         ...         ...       ...      ...           ...               ...   
95   FDBK-1095    CUST-109       Veg  MAA-CCU           4.0                 4   
96   FDBK-1096    CUST-105   Non-Veg  DEL-PNQ           1.0                 3   
97   FDBK-1097    CUST-109   Non-Veg  DEL-PNQ           5.0                 5   
98   FDBK-1098    CUST-104   Non-Veg  BOM-HYD           4.0                 3   
99   FDBK-1099    CUST-104       Veg  BOM-HYD           4.0                 4   

    packaging_rating compla